# Getting started

In [1]:
import pyobs
import numpy

pyobs.set_verbose('mfit') # prints additional information
pyobs.set_verbose('diff') # prints additional information

import matplotlib.pyplot as plt
%matplotlib widget

We create synthetic data from a model distribution that we try to fit using pyobs.

The model function is a simple polynomial $f = p_0 + p_1 x$

In [2]:
# true parameters
P1 = [2.3, 4.5]

The utility function `diff` provides automatically first and second derivatives, and is based on symbolic differentiation

In [3]:
xax1=[1,2,3,4,5,6,7,8]
[f1, df1, _] = pyobs.symbolic.diff('p0 + p1*x','x','p0,p1')

Now let's generate synthetic autocorrelated data without correlations, ie all points are uncorrelated: we do so by passing a `cov` matrix which is simply a vector. The function `acrandn` interprets it as a diagonal covariance matrix.

In [4]:
# generate autocorrelated data
N=500 # number of configs

# central values
mu=[f1(x,P1[0],P1[1]) for x in xax1]

# diagonal cov matrix
cov=[(mu[i]*0.2)**2 for i in range(len(mu))]

# autocorrelation time
tau=4.0

rng = pyobs.random.generator('fits')
data = rng.acrandn(mu,cov,tau,N)
data = numpy.reshape(data,numpy.size(data),)

Random generator initialized with seed = 2874984293 [fits]


In [5]:
yobs1 = pyobs.observable(description='test-data')
yobs1.create('A',data,shape=(len(mu),))
print(yobs1)

6.85(14)	11.35(27)	15.65(42)	19.79(49)	24.77(60)	28.37(55)	34.41(75)	37.53(92)



In [6]:
plt.figure()
[y,dy] = yobs1.error()
plt.errorbar(xax1,y,dy,fmt='.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

Now we create one instance of the `mfit` class. We need the kinematic coordinates `xax1`, the function and its derivatives obtained from `diff`, and the metric of the $\chi^2$, the matrix $W$, which is taken to be the inverse covariance matrix for a proper statistical interpretation of the $\chi^2$. In this case, since we have uncorrelated data points, we only needs their errors.

In [7]:
[y,dy] = yobs1.error()
W=1./dy**2
fit1 = pyobs.mfit(xax1,W,f1,df1)

The minimization of the $\chi^2$ is performed by simply calling the `mfit` class with the corresponding observable as argument. The output is new observable with the fit parameters.

In [8]:
pars = fit1(yobs1)
print('fit1 = ', pars)
print('Ratio with true parameters:',pars/numpy.array(P1))

chisquare = 3.7903256782960137
minimizer iterations = 3
minimizer status: Levenberg-Marquardt: reached  1.0e-08 per-cent tolerance on x0
mfit executed in 0.00345397 secs
fit1 =  2.43(17)	4.415(71)

Ratio with true parameters: 1.059(72)	0.981(16)



Using the fit parameters we can evaluate the fit function at arbitrary coordinates and check its compatibility with
the original data points, or extrapolate values from it.

In [9]:
plt.figure()
[y,dy] = yobs1.error()
plt.errorbar(xax1,y,dy,fmt='.',color='C0')

xax=numpy.arange(0,9,0.2)
yeval = fit1.eval(xax,pars)
[y,dy] = yeval.error()
plt.fill_between(xax,y+dy,y-dy,lw=0.1,alpha=0.5,color='C1')

[yzero] = fit1.eval(numpy.array([1.0]), pars)
print('extrapolated value at 1.0 = ', yzero)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

extrapolated value at 1.0 =  6.85(12)

